# 3차 과제
- 목적 : 변수 이름에 `종가`가 들어간 변수에 대해 평균, 분산과, 표준화 값 기준 1.96보다 큰 값들의 평균을 계산하는것을 목적으로 한다.
- 이때, 아래의 문제들을 순차적으로 해결하시오.


### 1) Pandas_2의 결과인 total_dat.xlsx을 읽고, 연도 변수를 추출하고, 2013년 1월 1일 이후 데이터만 추출하여 info 및 head의 결과를 출력하시오.

In [1]:
import pandas as pd
total_dat = pd.read_excel("total_dat.xlsx") # 상대경로로 설정했습니다. 현재 ipynb파일과 동일한 디렉토리 내에 total_dat.xlsx가 존재한다고 가정했습니다.
total_dat = total_dat.drop('Unnamed: 0',axis = 1) # unnamed: 0이라는 컬럼이 있으면 지웁니다.
total_dat['year'] = total_dat['date'].dt.year # year라는 새로운 컬럼 생성
after_2013 = total_dat['year'] >= 2013 

total_dat = total_dat.loc[after_2013 , :]
total_dat.info()
total_dat.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2481 entries, 248 to 2728
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2481 non-null   datetime64[ns]
 1   코스피        2481 non-null   float64       
 2   코스피200     2481 non-null   float64       
 3   삼성전자_종가    2481 non-null   int64         
 4   삼성전자_거래량   2481 non-null   int64         
 5   NAVER_종가   2481 non-null   int64         
 6   NAVER_거래량  2481 non-null   int64         
 7   year       2481 non-null   int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(4)
memory usage: 164.8 KB


,date,코스피,코스피200,삼성전자_종가,삼성전자_거래량,NAVER_종가,NAVER_거래량,year
248,2013-01-02,2031.10,269.16,1576000,229274,227500,154535,2013
249,2013-01-03,2019.41,267.52,1543000,284927,221000,356250,2013
250,2013-01-04,2011.94,266.14,1525000,260120,226500,234701,2013
251,2013-01-07,2011.25,265.97,1520000,252436,240000,404906,2013
252,2013-01-08,1997.94,263.95,1500000,276757,237000,198565,2013


### 2) 연도별로, 변수의 이름이 종가가 들어간 모든 변수들에 대해, 평균, 분산을 계산하시오.

In [2]:
positions = [i for i, col in enumerate(total_dat.columns) if '종가' in col]
end = total_dat.iloc[:, positions]

result = end.groupby(total_dat['year']).agg(['mean', 'var']) #result는 각 종가의 평균과 분산을 연도별로 계산한 데이터입니다.
result = result.reset_index()
result.columns = ['year','삼성전자_종가_평균', '삼성전자_종가_분산', 'NAVER_종가_평균', 'NAVER_종가_분산']

"""
또는, 명시적으로 컬럼명을 agg의 파라미터로 전달할 수 있습니다.
positions_and_names = [(i, col) for i, col in enumerate(total_dat.columns) if '종가' in col]

positions = []
names = []

for i, col in positions_and_names:
    positions.append(i)
    names.append(col)

result = total_dat.groupby('year')[[names[0], names[1]]].agg({names[0] : ['mean', 'var'], names[1] : ['mean', 'var']})
"""
result

,year,삼성전자_종가_평균,삼성전자_종가_분산,NAVER_종가_평균,NAVER_종가_분산
0,2013,1.428927e+06,7.502393e+09,387281.376518,2.716850e+10
1,2014,1.291796e+06,6.956860e+09,762493.877551,1.696808e+09
2,2015,1.299210e+06,1.088315e+10,618318.548387,5.103633e+09
3,2016,1.440004e+06,3.977535e+10,726626.016260,7.248096e+09
4,2017,2.320745e+06,7.997468e+10,813263.374486,1.670658e+09
5,2018,8.808141e+05,1.337890e+12,623284.836066,7.724934e+10
6,2019,4.658333e+04,1.425980e+07,138252.032520,4.270240e+08
7,2020,5.715323e+04,4.987001e+07,246167.338710,3.026683e+09
8,2021,7.915565e+04,2.329414e+07,389415.322581,1.143892e+09
9,2022,6.393537e+04,4.389878e+07,255689.024390,3.432730e+09


### 3) (2)에서 구하는 방법 혹은 (2)에서 구한 각 변수별 평균, 분산을 이용해 연도별로, 변수별로, 각각 표준화를 진행하고, 표준화된 데이터를 저장하시오.

In [3]:
def standardize(x,mean,var):
    return (x - mean)/(var**0.5)

join_dat = pd.merge(total_dat, result, on = "year", how = "left") # 표준화를 위해 total_dat과 result를 합쳤습니다.

standardized_data = pd.DataFrame()
standardized_data['year'] = join_dat['year']
standardized_data['삼성전자_standard'] = standardize(join_dat['삼성전자_종가'],join_dat['삼성전자_종가_평균'],join_dat['삼성전자_종가_분산'])
standardized_data['NAVER_standard'] = standardize(join_dat['NAVER_종가'],join_dat['NAVER_종가_평균'],join_dat['NAVER_종가_분산'])

standardized_data

,year,삼성전자_standard,NAVER_standard
0,2013,1.697980,-0.969378
1,2013,1.316990,-1.008813
2,2013,1.109177,-0.975445
3,2013,1.051451,-0.893542
4,2013,0.820548,-0.911743
...,...,...,...
2476,2023,1.166935,0.826571
2477,2023,1.370945,1.052256
2478,2023,1.656558,1.898574
2479,2023,1.126133,1.390783


### 4) (3)에서 구한 표준화 데이터에 대해 1.96보다 큰 값에 해당되는 종가 데이터들의 평균을 계산하시오.

In [4]:
def non_zero_mean(x):
    return(x[x != 0].mean())

standardized_data["over_196_삼성"] = (standardized_data['삼성전자_standard'] > 1.96) * join_dat['삼성전자_종가']
standardized_data["over_196_NAVER"] = (standardized_data['NAVER_standard'] > 1.96) * join_dat['NAVER_종가']


over_dat = standardized_data.groupby("year").agg({"over_196_삼성": non_zero_mean, "over_196_NAVER": non_zero_mean})
over_dat = over_dat.reset_index()
over_dat.columns = ["year","over_196_삼성","over_196_NAVER"]

over_dat

,year,over_196_삼성,over_196_NAVER
0,2013,NaN,727500.000000
1,2014,1.462333e+06,849166.666667
2,2015,NaN,775700.000000
3,2016,NaN,900000.000000
4,2017,NaN,911000.000000
5,2018,NaN,NaN
6,2019,5.569091e+04,181900.000000
7,2020,7.438889e+04,NaN
8,2021,8.986667e+04,NaN
9,2022,7.804545e+04,376000.000000


### 5) (2)의 결과와 (4)의 결과를 연도별로 결합하여 최종 결과를 출력하시오.

In [5]:
final_dat = pd.merge(result, over_dat, on = "year", how = "left")
final_dat

,year,삼성전자_종가_평균,삼성전자_종가_분산,NAVER_종가_평균,NAVER_종가_분산,over_196_삼성,over_196_NAVER
0,2013,1.428927e+06,7.502393e+09,387281.376518,2.716850e+10,NaN,727500.000000
1,2014,1.291796e+06,6.956860e+09,762493.877551,1.696808e+09,1.462333e+06,849166.666667
2,2015,1.299210e+06,1.088315e+10,618318.548387,5.103633e+09,NaN,775700.000000
3,2016,1.440004e+06,3.977535e+10,726626.016260,7.248096e+09,NaN,900000.000000
4,2017,2.320745e+06,7.997468e+10,813263.374486,1.670658e+09,NaN,911000.000000
5,2018,8.808141e+05,1.337890e+12,623284.836066,7.724934e+10,NaN,NaN
6,2019,4.658333e+04,1.425980e+07,138252.032520,4.270240e+08,5.569091e+04,181900.000000
7,2020,5.715323e+04,4.987001e+07,246167.338710,3.026683e+09,7.438889e+04,NaN
8,2021,7.915565e+04,2.329414e+07,389415.322581,1.143892e+09,8.986667e+04,NaN
9,2022,6.393537e+04,4.389878e+07,255689.024390,3.432730e+09,7.804545e+04,376000.000000
